In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re  # Para usar expresiones regulares
from datetime import datetime

# Función de Scraping de Productos
def scrape_products():
    # Configuración del ChromeDriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Si no quieres ver el navegador en acción
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar GPU (puede ser útil en máquinas sin GPU)
    chrome_options.add_argument("--no-sandbox")  # Evitar errores en entornos Linux (si es necesario)
    
    # Crear el servicio con el webdriver_manager
    service = Service(ChromeDriverManager().install())
    
    # Iniciar el navegador
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Página base para las URLs
    base_url = "https://www.hushpuppieskids.cl/nino/calzado?page="
    
    # Lista para almacenar los datos de los productos
    all_products_data = []

    # Empezamos con la página 1
    page_number = 1

    while True:
        # Generar la URL de la página actual
        url = base_url + str(page_number)
        driver.get(url)
        time.sleep(3)  # Esperar que cargue la página

        # Encontrar todos los productos en la página actual
        products = driver.find_elements(By.CSS_SELECTOR, "div.vtex-search-result-3-x-galleryItem")
        
        # Si no hay productos, significa que no hay más páginas
        if not products:
            print("No hay más productos. Terminando el scraping.")
            break
        
        # Extraer los datos de los productos
        for product in products:
            try:
                # Extraer el nombre del producto
                product_name = product.find_element(By.CSS_SELECTOR, ".vtex-product-summary-2-x-productNameContainer").text
                
                # Extraer el precio original (si existe)
                original_price_parts = product.find_elements(By.CSS_SELECTOR, ".vtex-product-price-1-x-listPriceValue span")
                if original_price_parts:
                    original_price = ''.join([part.text for part in original_price_parts])
                else:
                    original_price = None  # Si no existe un precio original
                
                # Extraer el precio final
                final_price_parts = product.find_elements(By.CSS_SELECTOR, ".vtex-product-price-1-x-sellingPriceValue span")
                final_price = ''.join([part.text for part in final_price_parts if part.text.strip() != ''])

                # Extraer el enlace de la imagen
                image_url = product.find_element(By.CSS_SELECTOR, "img").get_attribute("src")

                # Extraer la URL del producto
                product_link_element = product.find_element(By.CSS_SELECTOR, "a")
                product_url = product_link_element.get_attribute("href")
                
                # Obtener el SKU desde la URL de la imagen
                sku_match = re.search(r'arquivos/ids/(\d+)-', image_url)
                if sku_match:
                    sku = sku_match.group(1)  # Extraemos el SKU que está entre 'arquivos/ids/' y '-'
                else:
                    sku = None  # Si no se encuentra el SKU en la URL
                
                # Fecha de scraping (hoy)
                date_scraped = datetime.now().strftime('%Y-%m-%d')

                # Añadir los datos a la lista
                all_products_data.append({
                    "Nombre": product_name,
                    "Marca": None,  # No tenemos la marca
                    "Color 1": None,  # No tenemos el color
                    "SKU 1": sku,
                    "Precio Original": original_price,
                    "Precio Final": final_price,
                    "URL de la Imagen": image_url,
                    "URL Página": product_url,
                    "Fecha": date_scraped,
                    "0": None, "3": None, "6": None, "9": None, "0-6": None, "6-12": None,
                    "talla-unica": None, "chica": None, "grande": None, "15": None, "16": None,
                    "17": None, "18": None, "19": None, "3-6": None, "9-12": None, "18-24": None,
                    "24-36": None, "12": None, "20": None, "21": None, "22": None, "23": None,
                    "24": None, "25": None, "26": None, "27": None, "36": None, "0-1": None,
                    "1-2": None, "1-3": None, "1-4": None, "2": None, "2-3": None, "3-4": None,
                    "4": None, "4-6": None, "4-10": None, "6-8": None, "6-10": None, "8": None,
                    "8-10": None, "10": None, "10-12": None, "14": None, "28": None, "29": None,
                    "30": None, "31": None, "32": None, "150": None, "frutal": None, "citrico": None,
                    "33": None, "34": None, "35": None, "37": None, "38": None, "39": None, "40": None,
                    "41": None, "One": None, "GRUPO CATEGORIA": None, "CATEGORIA": None, "MUNDO": None,
                    "Descuento": None
                })
                
            except Exception as e:
                print(f"Error al extraer datos de un producto: {e}")
                continue
        
        # Incrementar el número de página
        page_number += 1

    # Cerrar el navegador
    driver.quit()

    # Crear un DataFrame con los datos y ordenarlos por las columnas especificadas
    column_order = [
        "Nombre", "Marca", "Color 1", "SKU 1", "Precio Original", "Precio Final", 
        "URL de la Imagen", "URL Página", "Fecha", "0", "3", "6", "9", "0-6", "6-12", 
        "talla-unica", "chica", "grande", "15", "16", "17", "18", "19", "3-6", "9-12", 
        "18-24", "24-36", "12", "20", "21", "22", "23", "24", "25", "26", "27", "36", 
        "0-1", "1-2", "1-3", "1-4", "2", "2-3", "3-4", "4", "4-6", "4-10", "6-8", "6-10", 
        "8", "8-10", "10", "10-12", "14", "28", "29", "30", "31", "32", "150", "frutal", 
        "citrico", "33", "34", "35", "37", "38", "39", "40", "41", "One", "GRUPO CATEGORIA", 
        "CATEGORIA", "MUNDO", "Descuento"
    ]
    
    df = pd.DataFrame(all_products_data)
    df = df[column_order]  # Reordenar las columnas

    # Guardar los datos en Excel usando el formato que prefieras
    df.to_excel(r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\ZAP3.xlsx', index=False)

    print("Datos guardados exitosamente en C:\\Users\\constanza.perez\\OneDrive - Colgram\\Escritorio\\HHH1.xlsx")

# Llamar a la función para hacer scraping
scrape_products()


No hay más productos. Terminando el scraping.
Datos guardados exitosamente en C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\HHH1.xlsx
